<a href="https://colab.research.google.com/github/Hoangnhat4869/TrafficLightDetection/blob/main/TrafficLightTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1. Install YOLOv5 and some requirements

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16620, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 16620 (delta 55), reused 63 (delta 29), pack-reused 16522
Receiving objects: 100% (16620/16620), 15.15 MiB | 28.99 MiB/s, done.
Resolving deltas: 100% (11408/11408), done.


In [ ]:
# import shutil
# shutil.rmtree("yolov5")

In [ ]:
cd yolov5

/content/yolov5


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

### 2. Train model

In [ ]:
import xml.etree.ElementTree as ET
import os
import shutil
from tqdm import tqdm
import pandas as pd
import numpy as np
from collections import defaultdict

In [ ]:
# create output path
output_path = '/content/yolov5'

# create train, val folders in data directory of YOLOv5
train_dir = output_path + '/data/train'
val_dir = output_path + '/data/valid'

if os.path.exists(train_dir): shutil.rmtree(train_dir)
if os.path.exists(val_dir): shutil.rmtree(val_dir)

os.mkdir(train_dir)
os.mkdir(val_dir)

# train, val each containing images and labels folders
os.mkdir(train_dir + '/images')
os.mkdir(train_dir + '/labels')
os.mkdir(val_dir + '/images')
os.mkdir(val_dir + '/labels')

In [ ]:
## copy traffic light images and labels to train and val folders
input_path = '/content/drive/MyDrive/TrafficLightDetection/data'

images_train_path = input_path + '/train/images'
labels_train_path = input_path + '/train/labels'

images_val_path = input_path + '/valid/images'
labels_val_path = input_path + '/valid/labels'

for file in sorted(os.listdir(images_train_path)):
  shutil.copy(images_train_path + '/' + file, train_dir + '/images')

for file in sorted(os.listdir(labels_train_path)):
  shutil.copy(labels_train_path + '/' + file, train_dir + '/labels')

for file in sorted(os.listdir(images_val_path)):
  shutil.copy(images_val_path + '/' + file, val_dir + '/images')

for file in sorted(os.listdir(labels_val_path)):
  shutil.copy(labels_val_path + '/' + file, val_dir + '/labels')

print(len(os.listdir(train_dir + '/images')))
print(len(os.listdir(train_dir + '/labels')))
print(len(os.listdir(val_dir + '/images')))
print(len(os.listdir(val_dir + '/labels')))

4557
4557
1020
1020


In [ ]:
classes = ['Green Light', 'Red Light', 'Speed Limit', 'Stop', 'No pedestrian crossing', 'No parking', 'Parking', 'Give way', 'One way', 'No left turn', 'Bus lane', 'Pedestrian crossing']
with open(f'{output_path}/data/data.yaml', 'w') as f:
  f.write('train: ./data/train/images\n')
  f.write('val: ./data/valid/images\n')
  f.write('nc: 12\n')
  f.write(f'names: {classes}')

In [ ]:
epochs = 30
!python train.py --img 640 --batch 32 --epochs {epochs} --data data.yaml --weights yolov5s.pt --cache

2024-05-22 05:39:05.946805: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 05:39:05.946856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 05:39:06.063278: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimi

### 3. Store weights

In [ ]:
import shutil
import os
respath = "/content/drive/MyDrive/TrafficLightDetection/weights"
if not os.path.exists(respath):
  os.mkdir(respath)
shutil.copy('/content/yolov5/runs/train/exp/weights/best.pt', '/content/drive/MyDrive/TrafficLightDetection/weights')

'/content/drive/MyDrive/TrafficLightDetection/weights/best.pt'